# Tests for calculating confidence interval

In [36]:
import scipy.stats as st
import math
import statsmodels as sm
import numpy as np

#### Z-test confidence interval

upper/lower bound of a confidence interval:<br>
X ± Z*(s/√n)<br>
X is the mean<br>
Z is the value from the standard normal distribution for the selected confidence level<br>
s is the standard deviation<br>
n is the number of observations<br>

A Z-test is any statistical test for which the distribution of the test statistic under the null hypothesis can be approximated by a normal distribution. Z-tests test the mean of a distribution. For each significance level in the confidence interval, the Z-test has a single critical value (for example, 1.96 for 5% two tailed).<br>
Z-test can be used when a sample size is big, usually > 30.

In [37]:
# Z-Test


def ztest(n, mean, std_dev, conf):
    '''
    This test assumes that data is normally distributed and works well for bigger number of samples (>30).
    Function takes number of samples (n), mean value (mean),
    standard deviation (std_dev) and confidence (conf).
    Returns lower and upper bounds for the confidence interval.
    '''
    z = st.norm.ppf(1-(1-conf)/2)
    upper_bound = mean + z*std_dev/math.sqrt(n)
    lower_bound = mean - z*std_dev/math.sqrt(n)
    
    return [lower_bound, upper_bound]

print(ztest(50, 50, 5, 0.95))


def reverse_ztest(mean, std_dev, diff, conf):
    '''
    Function takes mean of data (mean), standard deviation of data (std_dev),
    difference from mean to lower/upper bound which is upper_bound-mean or mean-lower_bound (diff)
    and confidence (conf).
    Returns rounded number of samples which should be taken to obtain a given confidence interval.
    '''
    z = st.norm.ppf(1-(1-conf)/2)
    n = (z*std_dev/(diff))**2
    return int(round(n))

print(reverse_ztest(50, 5, 1.39, 0.95))


[48.614096175650324, 51.385903824349676]
50


#### Z-test with precision
upper/lower bound of a confidence interval:<br>
X ± 10*Z*√(pr^2/n)<br>
X is the mean<br>
Z is the value from the standard normal distribution for the selected confidence level<br>
pr is the precision; the test assumes the worst possible precision = 0.5<br>
n is the number of observations<br>
An alternative way to calculate confidence intervals using z-test. When the standard deviation of the data is unknown, one can use this approach. Similarly, a sample size should be big ( > 30).

In [38]:
# Z-Test precission

def ztest_pr(n, mean, conf):
    '''
    This test assumes that data is normally distributed and works well for bigger number of samples (>30).
    Function takes number of samples (n), mean value (mean) and confidence (conf).
    Returns lower and upper bounds for the confidence interval.
    '''
    z = st.norm.ppf(1-(1-conf)/2)
    pr = z*math.sqrt(0.25/n)
    upper_bound = mean + 10*pr
    lower_bound = mean - 10*pr
    
    return [lower_bound, upper_bound]

print(ztest_pr(50, 50, 0.95))


def reverse_ztest_pr(mean, diff, conf):
    '''
    Function takes mean of data (mean), difference from mean to lower/upper bound which is upper_bound-mean 
    or mean-lower_bound (diff) and confidence (conf).
    Returns rounded number of samples which should be taken to obtain a given confidence interval.
    '''
    z = st.norm.ppf(1-(1-conf)/2)
    n = (z*math.sqrt(0.25)/(diff/10))**2
    return int(round(n))

print(reverse_ztest_pr(50, 1.39, 0.95))

[48.614096175650324, 51.385903824349676]
50


#### t-test confidence interval

upper/lower bound of a confidence interval:<br>
X ± t*(s/√n)<br>
X is the mean<br>
t is the value from the Student's t-distribution for the selected confidence level<br>
s is the standard deviation<br>
n is the number of observations<br>

The t-test is any statistical hypothesis test in which the test statistic follows a Student's t-distribution under the null hypothesis.
A t-test is the most commonly applied when the test statistic would follow a normal distribution if the value of a scaling term in the test statistic were known.<br> 
T-test can be used for a small number of samples ( < 30).

In [39]:
# T-test

def ttest(n, mean, std_dev, conf):
    '''
    This test works for smaller number of samples (<30), uses t-distribution instead the normal one.
    Function takes number of samples (n), mean value (mean), standard deviation and confidence (conf).
    Returns lower and upper bounds for the confidence interval.
    '''
    t = st.t.ppf(1-(1-conf)/2, n-1)
    upper_bound = mean + t*std_dev/math.sqrt(n)
    lower_bound = mean - t*std_dev/math.sqrt(n)
    
    return [lower_bound, upper_bound]

ttest(9, 50, 3, 0.95)


#def reverse_ttest()

[47.69399586496663, 52.30600413503337]

#### Loose test set bound (Langford)

In [40]:
# Loose test set bound Langford

def loose_langford(n, mean, conf):
    '''
    Function takes number of samples (n), mean value (mean) and confidence (conf).
    Returns lower and upper bounds for the confidence interval.
    '''
    pr = math.sqrt(math.log(2/(1-conf))/(n*2))
    upper_bound = mean + pr*10
    lower_bound = mean - pr*10
    return [lower_bound, upper_bound]

print(loose_langford(50, 50, 0.95))

def loose_langford_reverse(mean, diff, conf):
    '''
    Function takes mean of data (mean), difference from mean to lower/upper bound which is upper_bound-mean 
    or mean-lower_bound (diff) and confidence (conf).
    Returns rounded number of samples which should be taken to obtain a given confidence interval.
    '''
    n = math.log(2/(1-conf))/(2*(diff/10)**2)
    return int(round(n))

print(loose_langford_reverse(50, 1.92, 0.95))

[48.07935441736016, 51.92064558263984]
50


#### Clopper-Pearson (beta distribution)
The Clopper–Pearson interval is an early and very common method for calculating binomial confidence intervals. This is often called an 'exact' method, because it is based on the cumulative probabilities of the binomial distribution (i.e., exactly the correct distribution rather than an approximation). However, in cases where we know the population size, the intervals may not be the smallest possible.

In [41]:
# Clopper-Pearson

def clopper_pearson(n, mean, conf):
    '''
    Function takes number of samples (n), mean value (mean) and confidence (conf).
    Returns lower and upper bounds for the confidence interval.
    '''
    low, high = sm.stats.proportion.proportion_confint(n/2, n, alpha=1-conf, method = "beta")
    return [mean-(0.5-low)*10, mean+(high-0.5)*10]

print(clopper_pearson(50, 50, 0.95))

[48.55272997129908, 51.44727002870092]


#### Wilson score interval
The Wilson score interval is an improvement over the normal approximation interval in multiple respects. Unlike the symmetric normal approximation interval, the Wilson score interval is asymmetric. It does not suffer from problems of overshoot and zero-width intervals that afflict the normal interval, and it may be safely employed with small samples and skewed observations.<br>
The Wilson interval is derived from the Wilson Score Test, which belongs to a class of tests called Rao Score Tests. It relies on the asymptotic normality of your estimator.
Wilson intervals get their assymetry from the underlying likelihood function for the binomial, which is used to compute the "expected standard error" and "score" (i.e., first derivative of the likelihood function) under the null hypotheisis. Since these values will change as you very your null hypothesis, the interval where the normalized score (score/expected standard error) exceeds your pre-specified Z-cutoff for significance will not be symmetric, in general.

In [42]:
# Wilson score interval

def wilson(n, mean, conf):
    '''
    Function takes number of samples (n), mean value (mean) and confidence (conf).
    Returns lower and upper bounds for the confidence interval.
    '''
    low, high = sm.stats.proportion.proportion_confint(n/2, n, alpha=1-conf, method = "wilson")
    return [mean-(0.5-low)*10, mean+(high-0.5)*10]

print(wilson(50, 50, 0.95))

[48.664451431682856, 51.335548568317144]


#### Percentile Bootstrap Method
The percentile bootstrap interval is just the interval between the 100*(α/2) and 100*(1−α/2) percentiles of the distribution of θ estimates obtained from resampling, where θ represents a parameter of interest and α is the level of significance (e.g., α = 0.05 for 95% CIs) (Efron, 1982). A bootstrap percentile CI of θˆ (an estimator of θ) can be obtained as follows: (1) B random bootstrap samples are generated, (2) a parameter estimate is calculated from each bootstrap sample, (3) all B bootstrap parameter estimates are ordered from the lowest to highest, and (4) the CI is constructed as follows,<br>

$θˆ_{lower limit}$, $θˆ_{upper limit}$ = $θˆ_{j}$, $θˆ_{k}$ <br>

where $θˆ_{j}$ denotes the jth quantile (lower limit), and $θˆ_{k}$ denotes the kth quantile (upper limit); j=(α/2)*B, k=(1−α/2)*B. For example, a 95% percentile bootstrap CI with 1,000 bootstrap samples is the interval between the 25th quantile value and the 975th quantile value of the 1,000 bootstrap parameter estimates.

In [43]:
# Percentile bootstrap method

def percentile_BM(means, conf):
    '''
    Function takes list of resamples means obtained from bootstrap (means) and confidence (conf).
    Returns lower and upper bounds for the confidence interval.
    '''
    means.sort()
    means = np.array(means)
    lower_bound = np.percentile(means, 100*((1-conf)/2))
    upper_bound = np.percentile(means, 100*(conf + (1-conf)/2))
    return [lower_bound, upper_bound]


test_lst = list(np.random.normal(loc = 50, size=1000))
print(percentile_BM(test_lst, 0.95))

[48.11387887024648, 51.892874182382016]


#### Standard error method
We can only use the standard error rule when the bootstrap distribution is roughly normally shaped.

upper/lower bound of a confidence interval:<br>
X ± Z*SE<br>
X is the mean<br>
Z is the value from the standard normal distribution for the selected confidence level<br>
SE is the standard error (standard deviation of the resamples means)<br>

In [44]:
# Standard method error

def std_method(mean, means, conf):
    '''
    Function takes mean of the data (mean), list of resamples means obtained from bootstrap (means) and confidence (conf).
    Returns lower and upper bounds for the confidence interval.
    '''
    z = st.norm.ppf(1-(1-conf)/2)
    se = np.std(means)
    print(se)
    lower_bound = mean - z*se
    upper_bound = mean + z*se
    return [lower_bound, upper_bound]

print(std_method(50, test_lst, 0.95))

0.9926650022546512
[48.054412346867515, 51.945587653132485]
